In [1]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import block
import ecc
import helper
import network
import script
import tx

In [2]:
# Merkle Parent Example

from binascii import hexlify, unhexlify
from helper import double_sha256

tx_hash0 = unhexlify('c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5')
tx_hash1 = unhexlify('c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5')

merkle_parent = double_sha256(tx_hash0+tx_hash1)
hexlify(merkle_parent)

b'8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd'

In [13]:
reload(tx)
reload(ecc)
reload(helper)

from binascii import unhexlify
from io import BytesIO

raw_tx = unhexlify('0100000001a88d2263ec45751651480e9f9882dc9d1ff9c78a168589d4726422c8db2e406b000000006b483045022100b87a90a5d8ec1377d7a968e5aa191425b851997f5ee0438332e3473a2a3b35b502201f2aa45f948589dcb56e6ed2e357cb21733510d096cd61ae8bbd5d3d2bf01935412102bfaababf1ce5f56ec420dffdcfdb33cdc487eba7505ae6d7a8fe2bb89f61024cffffffff015ea82600000000001976a9140839c0f775141b1b7982891c1394a1c28278b8d288ac00000000')
stream = BytesIO(raw_tx)
t = tx.Tx.parse(stream)
input_index = 0
tx_in= t.tx_ins[input_index]
point = ecc.S256Point.parse(tx_in.sec_pubkey())
der, sighash = tx_in.der_signature()
sig = ecc.Signature.parse(der)
sha = helper.double_sha256(t.segwit_serialization(input_index, sighash))
z = int.from_bytes(sha, 'big')
point.verify(z, sig)

2535400
b'v\xa9\x14\xaa+\x85\xd4uc\xaf\x8e\xa5\xb1\xa1x\t\xcb(\xe6m\x0c\x9b\x00\x88\xac'


False

### Exercise 1

#### 1.1. Calculate the Merkle parent of these hashes:
```
f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0
3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181
```

#### 1.2. Make [this test](/edit/session7/helper.py) pass
```
helper.py:HelperTest:test_merkle_parent
```

In [3]:
# Exercise 1.1

from binascii import hexlify, unhexlify
from helper import double_sha256

hex_hash1 = 'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0'
hex_hash2 = '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181'

# unhexlify to get the bin hashes
hash1 = unhexlify(hex_hash1)
hash2 = unhexlify(hex_hash2)
# double_sha256 the combination
parent = double_sha256(hash1+hash2)
# hexlify to see the result
hexlify(parent)

b'7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800'

In [4]:
# Exercise 1.2

reload(helper)
run_test(helper.HelperTest('test_merkle_parent'))

.
----------------------------------------------------------------------
Ran 1 test in 0.020s

OK


In [5]:
# Merkle Parent Level Example

from binascii import hexlify, unhexlify
from helper import double_sha256, flip_endian, merkle_parent
hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
    'b13a750047bc0bdceb2473e5fe488c2596d7a7124b4e716fdd29b046ef99bbf0',
]
hashes = [unhexlify(x) for x in hex_hashes]
parent_level = []
for i in range(0, len(hex_hashes), 2):
    parent_level.append(merkle_parent(hashes[i], hashes[i+1]))
print([hexlify(x) for x in parent_level])

[b'8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd', b'7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800', b'11d16762bb5cfd315de66615182c9560aa2b00baca0b4a14aedd3b0c16779b1e']


### Exercise 2

#### 2.1. Calculate the next Merkle Parent Level given these hashes
```
8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd
7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800
ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7
68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069
43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27
4f492e893bf854111c36cb5eff4dccbdd51b576e1cfdc1b84b456cd1c0403ccb
```

#### 2.2. Make [this test](/edit/session7/helper.py) pass.
```
helper.py:HelperTest:test_merkle_parent_level0
```

In [6]:
# Exercise 2.1

from binascii import hexlify, unhexlify
from helper import merkle_parent

hex_hashes = [
    '8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd',
    '7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800',
    'ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7',
    '68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069',
    '43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27',
    '4f492e893bf854111c36cb5eff4dccbdd51b576e1cfdc1b84b456cd1c0403ccb',
]

# unhexlify to get all the hashes in binary
hashes = [unhexlify(h) for h in hex_hashes]
# initialize parent level
parent_level = []
# skip by two: use range(0, len(hashes), 2)
for i in range(0, len(hashes), 2):
    # calculate merkle_parent of i and i+1 hashes
    parent = merkle_parent(hashes[i], hashes[i+1])
    # print the hexlified hash
    print(hexlify(parent))
    # add parent to parent level
    parent_level.append(parent)

b'26906cb2caeb03626102f7606ea332784281d5d20e2b4839fbb3dbb37262dbc1'
b'717a0d17538ff5ad2c020bab38bdcde66e63f3daef88f89095f344918d5d4f96'
b'd20629030c7e48e778c1c837d91ebadc2f2ee319a0a0a461f4a9538b5cae2a69'


In [7]:
# Exercise 2.2

reload(helper)
run_test(helper.HelperTest('test_merkle_parent_level0'))

.
----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


In [8]:
# Odd Merkle Parent Level Example

from binascii import hexlify, unhexlify
from helper import double_sha256, flip_endian, merkle_parent
hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
]
hashes = [unhexlify(x) for x in hex_hashes]
if len(hashes) % 2 == 1:
    hashes.append(hashes[-1])
parent_level = []
for i in range(0, len(hex_hashes), 2):
    parent_level.append(merkle_parent(hashes[i], hashes[i+1]))
print([hexlify(x) for x in parent_level])

[b'8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd', b'7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800', b'3ecf6115380c77e8aae56660f5634982ee897351ba906a6837d15ebc3a225df0']


### Exercise 3

#### 3.1. Calculate the next Merkle Parent Level given these hashes
```
8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd
7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800
ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7
68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069
43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27
```

#### 3.2. Make [this test](/edit/session7/helper.py) pass.
```
helper.py:HelperTest:test_merkle_parent_level1
```

In [9]:
# Exercise 3.1

from binascii import hexlify, unhexlify
from helper import merkle_parent

hex_hashes = [
    '8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd',
    '7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800',
    'ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7',
    '68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069',
    '43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27',
]

# unhexlify to get all the hashes in binary
hashes = [unhexlify(h) for h in hex_hashes]
# if the number of hashes is odd, duplicate the last one
if len(hashes) % 2 == 1:
    hashes.append(hashes[-1])
# initialize parent level
parent_level = []
# skip by two: use range(0, len(hashes), 2)
for i in range(0, len(hashes), 2):
    # calculate merkle_parent of i and i+1 hashes
    parent = merkle_parent(hashes[i], hashes[i+1])
    # print the hexlified hash
    print(hexlify(parent))
    # add parent to parent level
    parent_level.append(parent)

b'26906cb2caeb03626102f7606ea332784281d5d20e2b4839fbb3dbb37262dbc1'
b'717a0d17538ff5ad2c020bab38bdcde66e63f3daef88f89095f344918d5d4f96'
b'd6c56a5281021a587f5a1e0dd4674bff012c69d960136d96e6d72261d5b696ae'


In [10]:
# Exercise 3.2

reload(helper)
run_test(helper.HelperTest('test_merkle_parent_level1'))

.
----------------------------------------------------------------------
Ran 1 test in 0.012s

OK


In [11]:
# Merkle Root Example

from binascii import hexlify, unhexlify
from helper import double_sha256, merkle_parent_level

hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
    '7d37b3d54fa6a64869084bfd2e831309118b9e833610e6228adacdbd1b4ba161',
    '8118a77e542892fe15ae3fc771a4abfd2f5d5d5997544c3487ac36b5c85170fc',
    'dff6879848c2c9b62fe652720b8df5272093acfaa45a43cdb3696fe2466a3877',
    'b825c0745f46ac58f7d3759e6dc535a1fec7820377f24d4c2c6ad2cc55c0cb59',
    '95513952a04bd8992721e9b7e2937f1c04ba31e0469fbe615a78197f68f52b7c',
    '2e6d722e5e4dbdf2447ddecc9f7dabb8e299bae921c99ad5b0184cd9eb8e5908',
    'b13a750047bc0bdceb2473e5fe488c2596d7a7124b4e716fdd29b046ef99bbf0',
]

current_level = [unhexlify(x) for x in hex_hashes]
while len(current_level) > 1:
    current_level = merkle_parent_level(current_level)
print(hexlify(current_level[0]))

b'acbcab8bcc1af95d8d563b77d24c3d19b18f1486383d75a5085c4e86c86beed6'


### Exercise 4

#### 4.1. Calculate the Merkle Root given these hashes
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 4.2. Make [this test](/edit/session7/helper.py) pass.
```
helper.py:HelperTest:test_merkle_root
```

In [12]:
# Exercise 4.1

from binascii import hexlify, unhexlify
from helper import double_sha256, merkle_parent_level
hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]

# unhexlify to get all the hashes in binary
hashes = [unhexlify(h) for h in hex_hashes]
# initialize current level to be the hashes
current_level = hashes
# loop until current_level has only 1 element
while len(current_level) > 1:
    # make the current level the parent level
    current_level = merkle_parent_level(current_level)
# print the hexlified root
print(hexlify(current_level[0]))

b'a67772634e542799333c6c98bc903e36b652918a8d8a9e069391c55b4276c8a1'


In [13]:
# Exercise 4.2

reload(helper)
run_test(helper.HelperTest('test_merkle_root'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [14]:
# Block Merkle Root Example

from binascii import hexlify, unhexlify
from helper import double_sha256, merkle_parent_level, merkle_root
tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
]
current_level = [unhexlify(x)[::-1] for x in tx_hex_hashes]
print(hexlify(merkle_root(current_level)[::-1]).decode('ascii'))

654d6181e18e4ac4368383fdc5eead11bf138f9b7ac1e15334e4411b3c4797d9


### Exercise 5

#### 5.1. Validate the merkle root for this block on Testnet:
Block Hash:
```
0000000000000451fa80fcdb243b84c35eaae215a85a8faa880559e8239e6f20
```

https://api.blockcypher.com/v1/btc/test3/blocks/0000000000000451fa80fcdb243b84c35eaae215a85a8faa880559e8239e6f20

Transaction Hashes:
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 5.2. Make [these tests](/edit/session7/block.py) pass.
```
block.py:BlockTest:test_validate_merkle_root
block.py:BlockTest:test_calculate_merkle_tree
```

In [15]:
# Exercise 5.1

from binascii import hexlify, unhexlify
from helper import double_sha256, flip_endian, merkle_root

tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]

# unhexlify and reverse ([::-1]) to get all the hashes in binary
hashes = [unhexlify(h)[::-1] for h in tx_hex_hashes]
# get the merkle root
root = merkle_root(hashes)
# hexlify the reversed root
print(hexlify(root[::-1]))

b'4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301'


In [16]:
# Exercise 5.2

reload(block)
run_test(block.BlockTest('test_validate_merkle_root'))
run_test(block.BlockTest('test_calculate_merkle_tree'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


In [17]:
# Merkle Path Example

import math

total_levels = math.ceil(math.log(16, 2))
merkle_path = []
index = 10
for i in range(total_levels):
    merkle_path.append(index)
    index = index // 2
print(merkle_path)

[10, 5, 2, 1]


### Exercise 6

#### 6.1. Get the Merkle Path for an item at index 13 (0-base) in 27 items.

#### 6.2. Make [this test](/edit/session7/helper.py) pass.
```
helper.py:HelperTest:test_merkle_path
```

In [18]:
# Exercise 6.1

import math

num_items = 27
index = 13

# get the total number of levels possible (math.ceil(math.log(num_items, 2)))
num_levels = math.ceil(math.log(num_items, 2))
# initialize merkle path array
merkle_path = []
# loop through number of levels
for _ in range(num_levels):
    # add the index to merkle path
    merkle_path.append(index)
    # index integer div by 2 to get the index at the next level
    index = index // 2
# print merkle path
print(merkle_path)

[13, 6, 3, 1, 0]


In [19]:
# Exercise 6.2

reload(helper)
run_test(helper.HelperTest('test_merkle_path'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [20]:
# Merkle Proof Example
from binascii import unhexlify

from block import Block
from helper import int_to_little_endian, merkle_path

hex_tx_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
    "f9dbfafc3af3400954975da24eb325e326960a25b87fffe23eef3e7ed2fb610e",
    "8e694f5092f6a644ab587ca445f9e949e4f5991d3c3c72bd4574a7c9896a2402",
    "9cc887977168f430f4f896dfc6fc7379834733ce938abe7cd8a1a668d1ea1841",
]
b = Block(
    version=536870914,
    prev_block=unhexlify('00000000000002dda81fd83ac5b944ad88592a213bfaf54bffad68725c782639'),
    merkle_root=unhexlify('f2710c8f3652ec6bfe79769458ae4be8117cad46964ce9dab9ce570bcb2ff9b0'),
    bits=int_to_little_endian(437256176, 4),
    nonce=b'\x00\x00\x00\x00',
    timestamp=1512503014,
    tx_hashes=[unhexlify(h) for h in hex_tx_hashes],
)
b.calculate_merkle_tree()
index, num_items = 10, 18
merkle_proof = []
for level, index_at_level in enumerate(merkle_path(index, num_items)):
    if index_at_level % 2 == 1:
        partner = index_at_level - 1
    else:
        partner = index_at_level + 1
    merkle_proof.append(b.merkle_tree[level][partner])
print(index, merkle_proof)

10 [b'8F\x80\x18[\xcb\x07\x9df\xce\xb8\xe6\x03\xab\xc2\xfe\xe9-<cuHu\xaf\xbd\x19P%[+\xa9\xb3', b'W@[\r\x17\xfeg\xaa\xf3\x9fn\x04\x17\x9d\xbd\x027x\xe1\x88K\xaa]\x14\t\xd0\xa7\xae\xa5\xc4\xa8\x9b', b'\x1a\xf7=\xdco\xdd"\xca\x92\x00\xd5\x19\xe1j,\xd4\xad#H\xb3\xe1.-Q\x8dv7=\xbe\x84\xf7\xab', b'\x18\xe5A6\x1eKE\x8ct\x9b\x90\xef\xcf\xc5\xf5pi\xc5\xdf\xbci\x07\xd2\x84:\xe0\xb9\x1b\xbb\xb6I6', b'K\xe0\x91\xa6\xab\xc8\xff;\xa7]\\L\x92b[\xbe\x8b\x01\xe9\xde\xfe\xda\x86+\x0f\xca@\x9dX1^\xa9']


### Exercise 7

#### 7.1. Create a Merkle Proof for this transaction
Transaction Hash:
```
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
```

Block Hex:
```
00000020fcb19f7895db08cadc9573e7915e3919fb76d59868a51d995201000000000000acbcab8bcc1af95d8d563b77d24c3d19b18f1486383d75a5085c4e86c86beed691cfa85916ca061a00000000
```

Transaction Hashes:
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 7.2. Make [this test](/edit/session7/block.py) pass.
```
block.py:BlockTest:test_create_merkle_proof
```

In [21]:
# Exercise 7.1

from io import BytesIO

from block import Block, Proof
from helper import merkle_path

tx_hash = unhexlify('e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208')
tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]
stream = BytesIO(unhexlify('00000020fcb19f7895db08cadc9573e7915e3919fb76d59868a51d995201000000000000acbcab8bcc1af95d8d563b77d24c3d19b18f1486383d75a5085c4e86c86beed691cfa85916ca061a00000000'))
b = Block.parse(stream)
b.tx_hashes = [unhexlify(x) for x in tx_hex_hashes]

# calculate the merkle tree first
b.calculate_merkle_tree()
# get the index of the tx_hash in the list of tx_hashes (b.tx_hashes.index(tx_hash))
index = b.tx_hashes.index(tx_hash)
# initialize the proof hashes
proof_hashes = []
# loop through the enumerated merkle path (keep track of level and index at level)
for level, index_at_level in enumerate(merkle_path(index, len(b.tx_hashes))):
    # if the current level index is odd, the partner is - 1
    if index_at_level % 2 == 1:
        partner = index_at_level - 1
    # if the current level index is even, the partner is + 1
    else:
        partner = index_at_level + 1
    # add the partner to proof hashes
    merkle_proof.append(b.merkle_tree[level][partner])
print(index, merkle_proof)

6 [b'8F\x80\x18[\xcb\x07\x9df\xce\xb8\xe6\x03\xab\xc2\xfe\xe9-<cuHu\xaf\xbd\x19P%[+\xa9\xb3', b'W@[\r\x17\xfeg\xaa\xf3\x9fn\x04\x17\x9d\xbd\x027x\xe1\x88K\xaa]\x14\t\xd0\xa7\xae\xa5\xc4\xa8\x9b', b'\x1a\xf7=\xdco\xdd"\xca\x92\x00\xd5\x19\xe1j,\xd4\xad#H\xb3\xe1.-Q\x8dv7=\xbe\x84\xf7\xab', b'\x18\xe5A6\x1eKE\x8ct\x9b\x90\xef\xcf\xc5\xf5pi\xc5\xdf\xbci\x07\xd2\x84:\xe0\xb9\x1b\xbb\xb6I6', b'K\xe0\x91\xa6\xab\xc8\xff;\xa7]\\L\x92b[\xbe\x8b\x01\xe9\xde\xfe\xda\x86+\x0f\xca@\x9dX1^\xa9', b'\x9e\xd0\xa5C\x0b[S\x08"\xb1\xce\x1b+\x9a\x03\xd5\x13\xc8\x88\xaa\xa3\xf0(\xf0A\xbf\x14>\xfd\x8c\x1b\x92', b"\x1d\xc4\xb48\xb3\xa8B\xbc\xddF\xb6\xeaZJ\xac\x8df\xbe\x85\x8b\x0b\xa4\x12W\x80'\xa1\xf1\xfe\x83\x8cQ", b'\x15o6b\xb0z\xaaJ\r\x97b\xfa\xaa\x8c\x18\xaf\xe4\xc2\x11\xff\x92\xeb\x1e\xae\x19R\xaafb{\xbf.', b'RL\x93\xc6\xdd\x08t\xc5\xfd\x9eNW\xcf\xe81v\xe3\xc2\x84\x1c\x97:\xfb@C\xd2%\xc2,\xc5)\x83']


In [22]:
# Exercise 7.2

reload(block)
run_test(block.BlockTest('test_create_merkle_proof'))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 8

#### 8.1. Verify a Merkle Proof
Transaction Hash:
```
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
```

Index:
```
6
```

Merkle Root:
```
4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301
```

Merkle Proof:
```
9ed0a5430b5b530822b1ce1b2b9a03d513c888aaa3f028f041bf143efd8c1b92
1dc4b438b3a842bcdd46b6ea5a4aac8d66be858b0ba412578027a1f1fe838c51
156f3662b07aaa4a0d9762faaa8c18afe4c211ff92eb1eae1952aa66627bbf2e
524c93c6dd0874c5fd9e4e57cfe83176e3c2841c973afb4043d225c22cc52983
```

#### 8.2. Make [this test](/edit/session7/block.py) pass.
```
block.py:BlockTest:test_verify_merkle_proof
```

In [23]:
# Exercise 8.1

from binascii import hexlify, unhexlify
from block import Proof
from helper import double_sha256, flip_endian, merkle_root, merkle_path, merkle_parent

tx_hash = unhexlify('e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208')
merkle_root = unhexlify('4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301')
proof_hex_hashes = [
    '9ed0a5430b5b530822b1ce1b2b9a03d513c888aaa3f028f041bf143efd8c1b92',
    '1dc4b438b3a842bcdd46b6ea5a4aac8d66be858b0ba412578027a1f1fe838c51',
    '156f3662b07aaa4a0d9762faaa8c18afe4c211ff92eb1eae1952aa66627bbf2e',
    '524c93c6dd0874c5fd9e4e57cfe83176e3c2841c973afb4043d225c22cc52983',
]
proof_hashes = [unhexlify(x) for x in proof_hex_hashes]
index = 6

# get the current hash (reverse it as it needs to be little endian)
current = tx_hash[::-1]
# get the merkle path for the index. assume the num items is 2**len(proof_hashes)
path = merkle_path(6, 2**len(proof_hashes))
# loop through proof hashes
for proof_hash, index_at_level in zip(proof_hashes, path):
    # if index_at_level is odd, proof_hash is on the left
    if index_at_level % 2 == 1:
        current = merkle_parent(proof_hash, current)
    # if index_at_level is even, proof_hash is on the right
    else:
        current = merkle_parent(current, proof_hash)
# see if the current one reversed is the same as merkle root
print(current[::-1] == merkle_root)

True


In [24]:
# Exercise 8.2

reload(block)
run_test(block.BlockTest('test_verify_merkle_proof'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 9

#### 9.1. Parse this message
```
f9beb4d976657261636b000000000000000000005df6e0e2
```

#### 9.2. Make [these test](/edit/session7/network.py) pass.
```
network.py:NetworkTest:test_parse
network.py:NetworkTest:test_serialize
```

In [25]:
# Exercise 9.1

from binascii import unhexlify

msg = unhexlify('f9beb4d976657261636b000000000000000000005df6e0e2')

# first 4 are network magic
magic = msg[:4]
# next 12 are command
command = msg[4:16]
# next 4 are payload length
payload_length = msg[16:20]
# next 4 are checksum
checksum = msg[20:24]
# rest is payload
payload = msg[24:]
print(command)

b'verack\x00\x00\x00\x00\x00\x00'


In [26]:
# Exercise 9.2

reload(network)
run_test(network.NetworkEnvelopeTest('test_parse'))
run_test(network.NetworkEnvelopeTest('test_serialize'))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 10

#### 10.1. Parse this message
```
f9beb4d974780000000000000000000002010000e293cdbe01000000016dbddb085b1d8af75184f0bc01fad58d1266e9b63b50881990e4b40d6aee3629000000008b483045022100f3581e1972ae8ac7c7367a7a253bc1135223adb9a468bb3a59233f45bc578380022059af01ca17d00e41837a1d58e97aa31bae584edec28d35bd96923690913bae9a0141049c02bfc97ef236ce6d8fe5d94013c721e915982acd2b12b65d9b7d59e20a842005f8fc4e02532e873d37b96f09d6d4511ada8f14042f46614a4c70c0f14beff5ffffffff02404b4c00000000001976a9141aa0cd1cbea6e7458a7abad512a9d9ea1afb225e88ac80fae9c7000000001976a9140eab5bea436a0484cfab12485efda0b78b4ecc5288ac00000000
```


In [27]:
# Exercise 10.1

from io import BytesIO

from network import NetworkEnvelope

stream = BytesIO(unhexlify('f9beb4d974780000000000000000000002010000e293cdbe01000000016dbddb085b1d8af75184f0bc01fad58d1266e9b63b50881990e4b40d6aee3629000000008b483045022100f3581e1972ae8ac7c7367a7a253bc1135223adb9a468bb3a59233f45bc578380022059af01ca17d00e41837a1d58e97aa31bae584edec28d35bd96923690913bae9a0141049c02bfc97ef236ce6d8fe5d94013c721e915982acd2b12b65d9b7d59e20a842005f8fc4e02532e873d37b96f09d6d4511ada8f14042f46614a4c70c0f14beff5ffffffff02404b4c00000000001976a9141aa0cd1cbea6e7458a7abad512a9d9ea1afb225e88ac80fae9c7000000001976a9140eab5bea436a0484cfab12485efda0b78b4ecc5288ac00000000'))
NetworkEnvelope.parse(stream)

tx          : 01000000016dbddb085b1d8af75184f0bc01fad58d1266e9b63b50881990e4b40d6aee3629000000008b483045022100f3581e1972ae8ac7c7367a7a253bc1135223adb9a468bb3a59233f45bc578380022059af01ca17d00e41837a1d58e97aa31bae584edec28d35bd96923690913bae9a0141049c02bfc97ef236ce6d8fe5d94013c721e915982acd2b12b65d9b7d59e20a842005f8fc4e02532e873d37b96f09d6d4511ada8f14042f46614a4c70c0f14beff5ffffffff02404b4c00000000001976a9141aa0cd1cbea6e7458a7abad512a9d9ea1afb225e88ac80fae9c7000000001976a9140eab5bea436a0484cfab12485efda0b78b4ecc5288ac00000000

### Exercise 11

Use https://bitnodes.earn.com/nodes/ to find nodes to connect to.

#### 11.1. Run the code below and parse some messages

In [28]:
import socket
from binascii import unhexlify, hexlify
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
msg = unhexlify('f9beb4d976657273696f6e0000000000650000005f1a69d2721101000100000000000000bc8f5e5400000000010000000000000000000000000000000000ffffc61b6409208d010000000000000000000000000000000000ffffcb0071c0208d128035cbc97953f80f2f5361746f7368693a302e392e332fcf05050001')
s.connect(('46.101.99.121', 8333))
s.sendall(msg)
data = s.recv(200)
print(hexlify(data))

b'f9beb4d976657273696f6e00000000006800000019dd02907f1101000d00000000000000e001275a00000000010000000000000000000000000000000000ffffc61b6409208d0d00000000000000000000000000000000000000000000000000553c52a904d620a1122f5361746f7368693a302e31352e302e312f7b98070001f9beb4d976657261636b000000000000000000005df6e0e2'
